## Experiment 3
<br>
-- Combination of raw numbers and katakana characters



In [1]:
import json
import utils
import data_handler as dh

from collections import defaultdict
from itertools import product
from IPython.display import clear_output

import pandas as pd
import numpy as np

### Data setting

In [2]:
data_path = '/home/donghyun/eye_writing_classification/v2_dataset/200_points_dataset/'

with open(data_path + 'eog_raw_numbers_200.json') as f:
  eog_raw_numbers = json.load(f)

with open(data_path + 'eog_katakana_200.json') as f:
  eog_katakana = json.load(f)

with open(data_path + 'reference_data_200.json') as f:
  reference_data = json.load(f)

In [4]:
print(eog_katakana.keys())
print(eog_raw_numbers.keys())

dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'])
dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])


In [ ]:
# katakana class를 10~21으로 치환 후 merge

katakana_le = {'1':'10', '2':'11', '3':'12', '4':'13', '5':'14',
               '6':'15', '7':'16', '8':'17', '9':'18', '10':'19',
               '11':'20', '12':'21'}

all_data = eog_raw_numbers.copy()
for k in eog_katakana.keys():
    cnvt_key = katakana_le[k]
    all_data[cnvt_key] = eog_katakana[k]

all_ref = defaultdict()
all_ref['all'] = reference_data['numbers'].copy()
for k in reference_data['katakana'].keys():
    cnvt_key = katakana_le[k]
    all_ref['all'][cnvt_key] = reference_data['katakana'][k]


print(all_data.keys())
print(np.array(all_data['0']).shape)
print(np.array(all_data['11']).shape)
print(all_ref.keys())
print(all_ref['all'].keys())
print(np.array(all_ref['all']['0']).shape)
print(np.array(all_ref['all']['11']).shape)

In [ ]:
save_path = '/home/donghyun/eye_writing_classification/experiments/save/'

best_perform_df = pd.read_csv(save_path + 'experiment2_vit_hyperparams.csv')

### Experiment

In [ ]:
# Config class

class ViT_Config:
    split_ratio = 0.3
    ref_key = 'all'
    batch_size = 22            # fix : must be equaled with number of test pairs 
    n_batch = 50
    model_type = 'ViTBaseModel'
    ViT_params = {}
    epochs = 1000


class Hyb_Config:
    split_ratio = 0.3
    ref_key = 'all'
    batch_size = 16               # fix : Not must be equaled with number of test pairs 
    n_batch = 50
    model_type = 'HybridBaseModel'
    epochs = 1000

In [ ]:
vit_cfg = ViT_Config()
hyb_cfg = Hyb_Config()

vit_cfg = ViT_Config()
vit_cfg.ViT_params = best_perform_df.loc[0]
vit_cfg.ViT_params['mlp_units'] = list(map(int, vit_cfg.ViT_params['mlp_units']))

vit_dict = defaultdict(list)
bybrid_dict = defaultdict(list)

times = 10
for t in range(times):

    _, vit_train_acc, vit_train_loss, vit_test_acc = utils.experiment(vit_cfg, all_data, all_ref)
    _, hyb_train_acc, hyb_train_loss, hyb_test_acc = utils.experiment(hyb_cfg, all_data, all_ref)

    vit_dict[t] = [vit_train_acc, vit_train_loss, vit_test_acc]
    bybrid_dict[t] = [hyb_train_acc, hyb_train_loss, hyb_test_acc]


clear_output()

### Save

In [ ]:
import json

save_path = '/home/donghyun/eye_writing_classification/experiments/save/'

with open(save_path + 'experiment3_ViT_results.json', 'w') as f:
    json.dump(dict(vit_dict),f)

with open(save_path + 'experiment3_Hybrid_results.json', 'w') as f:
    json.dump(dict(bybrid_dict),f)

### Visualization

In [ ]:
save_path = '/home/donghyun/eye_writing_classification/experiments/save/'

vit_dict = json.dumps(save_path + 'experiment3_ViT_results.json')
bybrid_dict = json.dumps(save_path + 'experiment3_Hybrid_results.json')